In [1]:
import numpy as np 
import pandas as pd 
import re  
import nltk 
from nltk.corpus import stopwords
import AirBnB as ab
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
# Read in the reviews.csv file. 

file_path="./reviews.csv"
reviews_df=ab.read_file(file_path) 
five_lines=ab.get_five_lines(reviews_df)

In [3]:
# Read in the reviews.csv file. 
file_path="./listings.csv"
listings_df=ab.read_file(file_path) 
five_lines=ab.get_five_lines(listings_df)

In [4]:
# Checking and cleaning comments inside the review file
cols=['listing_id','reviewer_id','reviewer_name','comments']
listing_reviews_df=reviews_df[cols]
listing_reviews_df.shape
reviews_df.dtypes
reviews_df['comments']=reviews_df['comments'].astype(str)
reviews_df["review_clean"] = reviews_df["comments"].apply(lambda x: ab.clean_text(x))
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,review_clean
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...,cute cozy place perfect location everything
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...,kelly great room central location \r\nbeautifu...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb...",spacious apartment great neighborhood kind apa...
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...,close seattle center offer ballet theater muse...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...,kelly great host accommodate great neighborhoo...


In [5]:
#Getting the listings and review scores
cols=['id','review_scores_rating']
listings_df=listings_df[cols]
listings_df=listings_df.rename(columns={"id":'listing_id'})

In [6]:
# Binning the review scores and figuring out the buckets to create score_sentiment column
listings_reviews_df = pd.merge(left=listings_df, right=reviews_df, left_on='listing_id', right_on='listing_id')
listings_reviews_df.shape
listings_reviews_df['review_scores_rating']=listings_reviews_df['review_scores_rating'].astype('float')
listings_reviews_df.head()
listings_reviews_df['review_scores_rating_bins']=pd.cut(listings_reviews_df.review_scores_rating,bins = [0,85.0,90.0,95.0,97.0,100.0])
listings_reviews_df.head()
listings_reviews_df['review_scores_rating_bins'].value_counts().sort_index()
listings_reviews_df['score_sentiment']=pd.cut(listings_reviews_df.review_scores_rating,bins = [0,85.0,90.0,95.0,97.0,100.0],labels=['Bad','Fair','Good','Average','Best'])
listings_reviews_df.head()


,listing_id,review_scores_rating,id,date,reviewer_id,reviewer_name,comments,review_clean,review_scores_rating_bins,score_sentiment
0,241032,95.0,682061,2011-11-01,479824,Bro,Excellent all the way around. \r\n\r\nMaija wa...,excellent way around \r\n\r\nmaija excellent h...,"(90.0, 95.0]",Good
1,241032,95.0,691712,2011-11-04,357699,Megan,Maija's apartment was a wonderful place to sta...,maija's apartment wonderful place stay extreme...,"(90.0, 95.0]",Good
2,241032,95.0,702999,2011-11-08,1285567,Marylee,one of the most pleasant stays i've had in my ...,one pleasant stay i've travel maija wonderful ...,"(90.0, 95.0]",Good
3,241032,95.0,717262,2011-11-14,647857,Graham,"Maija's suite is beautiful, cozy and convenien...",maija's suite beautiful cozy conveniently loca...,"(90.0, 95.0]",Good
4,241032,95.0,730226,2011-11-19,1389821,Franka,Our stay was short and pleasant. With its own ...,stay short pleasant porch space flat boost lot...,"(90.0, 95.0]",Good


In [7]:
# Checking for nulls in score_sentiment
listings_reviews_df['score_sentiment'].isnull().sum()
listings_reviews_df.dropna(subset=['score_sentiment'], how='all', inplace=True)
listings_reviews_df.shape
listings_reviews_df['score_sentiment'].isnull().sum()
listings_reviews_df.shape

(84829, 10)

In [8]:
# Running the model and getting the predictions
predictions, confusion_matrix, classification_report, accuracy_score = ab.clean_fit_tfdf_random_forest(listings_reviews_df, 'review_clean', 'score_sentiment', 0.4, 0, 100, 2000, 5, 0.7)

In [9]:
# printing out the output from the modelling
print(confusion_matrix)  

[[3322    8 1254   16 4510]
 [  65  150   59   46  883]
 [ 824    8 3997   18 4048]
 [ 217   34  216  224 1977]
 [ 940   11 1185   42 9878]]
[[3322    8 1254   16 4510]
 [  65  150   59   46  883]
 [ 824    8 3997   18 4048]
 [ 217   34  216  224 1977]
 [ 940   11 1185   42 9878]]
0.5178297772014617


In [10]:
print(classification_report)

[[3322    8 1254   16 4510]
 [  65  150   59   46  883]
 [ 824    8 3997   18 4048]
 [ 217   34  216  224 1977]
 [ 940   11 1185   42 9878]]


In [11]:
print(accuracy_score)

0.5178297772014617
